In [2]:
# Andy Vo
# ECGR 4106 Homework 3
# Due: 3/29/2022

In [3]:
# For problem 1 parts a and b

%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
import collections

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.set_printoptions(edgeitems=2)
torch.manual_seed(123)

In [4]:
class_names = ['airplane','automobile','bird','cat','deer',
               'dog','frog','horse','ship','truck']

In [5]:
from torchvision import datasets
from torchvision import transforms

# Downloading CIFAR
data_path = '/content/drive/MyDrive/Colab Notebooks/ECGR_4106_Homework/Homework1/Text_Files/'
cifar10 = datasets.CIFAR10(
    data_path, train=True, download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4915, 0.4823, 0.4468),
                             (0.2470, 0.2435, 0.2616))
    ]))

Files already downloaded and verified


In [6]:
# convert PIL Images to pytorch tensor
cifar10_val = datasets.CIFAR10(
    data_path, train=False, download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4915, 0.4823, 0.4468),
                             (0.2470, 0.2435, 0.2616))
    ]))


Files already downloaded and verified


In [7]:
# Initialize for all 10 classes
# Dataset subclass that includes all 10 classes



label_map = {0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 8, 9: 9}

class_names = ['airplane','automobile','bird','cat','deer',
               'dog','frog','horse','ship','truck']

cifar10_train = [(img, label_map[label])
          for img, label in cifar10
          if label in [0, 1, 2, 3, 4, 5, 6, 7, 8 ,9]]

cifar10_test  = [(img, label_map[label])
              for img, label in cifar10_val
              if label in [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]]


#cifar10_train = [(img, label)
#         for img, label in cifar10]
#cifar10_test = [(img, label)
#              for img, label in cifar10_val]

In [8]:
# Move things to the GPU if one if available
device = (torch.device('cuda') if torch.cuda.is_available()
else torch.device('cpu'))

print(f"Training on device {device}.")

Training on device cpu.


In [9]:
# Connected hidden Layer 
connected_model = nn.Sequential(
            nn.Linear(3072, 1024),
            nn.Tanh(),
            nn.Linear(1024, 512),
            nn.Tanh(),
            nn.Linear(512, 128),
            nn.Tanh(),
            nn.Linear(128, 10))

In [10]:
numel_list = [p.numel()
              for p in connected_model.parameters()
              if p.requires_grad == True]
sum(numel_list), numel_list

(3738506, [3145728, 1024, 524288, 512, 65536, 128, 1280, 10])

In [11]:
# First hidden layer
first_model = nn.Sequential(
                nn.Linear(3072, 512),
                nn.Tanh(),
                nn.Linear(512, 10),
                nn.LogSoftmax(dim=1))

first_model.to(device)


Sequential(
  (0): Linear(in_features=3072, out_features=512, bias=True)
  (1): Tanh()
  (2): Linear(in_features=512, out_features=10, bias=True)
  (3): LogSoftmax(dim=1)
)

In [12]:
numel_list = [p.numel() for p in first_model.parameters()]
sum(numel_list), numel_list

(1578506, [1572864, 512, 5120, 10])

In [13]:
linear = nn.Linear(3072, 1024)

linear.weight.shape, linear.bias.shape

(torch.Size([1024, 3072]), torch.Size([1024]))

In [14]:
# use Conv2d for images
conv = nn.Conv2d(3, 16, kernel_size=3) # <1>
conv

Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1))

In [15]:
conv.weight.shape, conv.bias.shape

(torch.Size([16, 3, 3, 3]), torch.Size([16]))

In [16]:
img, _ = cifar10_train[0]
output = conv(img.unsqueeze(0))
img.unsqueeze(0).shape, output.shape

(torch.Size([1, 3, 32, 32]), torch.Size([1, 16, 30, 30]))

In [17]:
conv = nn.Conv2d(3, 1, kernel_size=3, padding=1) # <1>
output = conv(img.unsqueeze(0))
img.unsqueeze(0).shape, output.shape

(torch.Size([1, 3, 32, 32]), torch.Size([1, 1, 32, 32]))

In [18]:
with torch.no_grad():
    conv.bias.zero_()
    
with torch.no_grad():
    conv.weight.fill_(1.0 / 9.0)

In [19]:
conv = nn.Conv2d(3, 1, kernel_size=3, padding=1)

with torch.no_grad():
    conv.weight[:] = torch.tensor([[-1.0, 0.0, 1.0],
                                   [-1.0, 0.0, 1.0],
                                   [-1.0, 0.0, 1.0]])
    conv.bias.zero_()

In [20]:
# it takes as input the size of the neighborhood over which to operate the pooling operation.
pool = nn.MaxPool2d(2)
output = pool(img.unsqueeze(0))

img.unsqueeze(0).shape, output.shape

(torch.Size([1, 3, 32, 32]), torch.Size([1, 3, 16, 16]))

In [21]:
# another convolution that generates an 8-channel 16 × 16 output
# Must modify this part for both parts of a and b

# first run with a single convulutional layer
# second run with an additional convolutional layer


model = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.Tanh(),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.Tanh(),
            nn.MaxPool2d(2),
            nn.BatchNorm2d(64),

            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.Tanh(),
            nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1),
            nn.Tanh(),
            nn.MaxPool2d(2),
            nn.BatchNorm2d(128),

            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
            nn.Tanh(),
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.Tanh(),
            nn.MaxPool2d(2),
            nn.BatchNorm2d(256),

            nn.Flatten(),
            nn.Linear(8 * 8 * 8, 32),
            nn.Tanh(),
            nn.Linear(1024, 512),
            nn.Tanh(),
            nn.Linear(32, 10))

def forward(self, xb):
    return self.network(xb)

In [22]:
numel_list = [p.numel() for p in model.parameters()]
sum(numel_list), numel_list

(1668522,
 [864,
  32,
  18432,
  64,
  64,
  64,
  73728,
  128,
  147456,
  128,
  128,
  128,
  294912,
  256,
  589824,
  256,
  256,
  256,
  16384,
  32,
  524288,
  512,
  320,
  10])

In [23]:
# modules for added flexibility by subclassing nn.Module

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.act1 = nn.Tanh()
        self.pool1 = nn.MaxPool2d(2)
        self.conv2 = nn.Conv2d(16, 8, kernel_size=3, padding=1)
        self.act2 = nn.Tanh()
        self.pool2 = nn.MaxPool2d(2)
        self.fc1 = nn.Linear(8 * 8 * 8, 32)
        self.act3 = nn.Tanh()
        self.fc2 = nn.Linear(32, 2)

    def forward(self, x):
        out = self.pool1(self.act1(self.conv1(x)))
        out = self.pool2(self.act2(self.conv2(out)))
        out = out.view(-1, 8 * 8 * 8) # <1>
        out = self.act3(self.fc1(out))
        out = self.fc2(out)
        return out

In [24]:
model = Net()

numel_list = [p.numel() for p in model.parameters()]
sum(numel_list), numel_list

(18090, [432, 16, 1152, 8, 16384, 32, 64, 2])

In [25]:
# take inputs and parameters as arguments to the function call.
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()

In [26]:
model = Net()
model(img.unsqueeze(0))

tensor([[-0.0376,  0.1191, -0.0832, -0.0974,  0.1225, -0.0004,  0.0980, -0.0162,
         -0.0362,  0.1067]], grad_fn=<AddmmBackward0>)

In [27]:
# Used the datetime module included with Python
import datetime  # <1>

# Loop over the epochs, numbered from 1 to n_epochs rather than starting at 0 
def training_loop(n_epochs, optimizer, model, loss_fn, train_loader):
    for epoch in range(1, n_epochs + 1):  # <2>
        loss_train = 0.0
        
        # Loops over our dataset in the batches the loader creates
        for imgs, labels in train_loader:  # <3>
            
            # Feeds a batch thrugh out model
            #imgs, labels = imgs.to(device), labels.to(device)

            outputs = net(imgs)
            #outputs = first_model(imgs.view(imgs.shape[0], -1))

            
            # Computes the loss being minimized
            loss = loss_fn(outputs, labels)  # <5>

            optimizer.zero_grad()  # <6>
            
            loss.backward()  # <7> Performs the backward step
            
            optimizer.step()  # <8>

            loss_train += loss.item()  # <9>

        if epoch == 1 or epoch % 10 == 0:
            print('{} Epoch {}, Training loss {}'.format(
                datetime.datetime.now(), epoch,
                loss_train / len(train_loader)))  # <10> 
                # divides by the length of the trainng data loader to get avg

In [45]:
# Batches up the examples of our cifar10 dataset
# Shuffling randomizes the order of the examples from dataset
train_loader = torch.utils.data.DataLoader(cifar10_train, batch_size=64,
                                           shuffle=True)  # <1>

model = Net()  #  <2>
optimizer = optim.SGD(model.parameters(), lr=1e-2)  #  <3>
loss_fn = nn.CrossEntropyLoss()  #  <4>

training_loop(  # <5>
    n_epochs = 300,
    optimizer = optimizer,
    model = model,
    loss_fn = loss_fn,
    train_loader = train_loader,
    
)

KeyboardInterrupt: ignored

In [37]:
train_loader = torch.utils.data.DataLoader(cifar10, batch_size=64,
                                           shuffle=False)
val_loader = torch.utils.data.DataLoader(cifar10_val, batch_size=64,
                                         shuffle=False)

all_acc_dict = collections.OrderedDict()


def validate(model, train_loader, val_loader):
    for name, loader in [("train", train_loader), ("val", val_loader)]:
        correct = 0
        total = 0

        # not update the parameters b/c we do not want gradients here
        with torch.no_grad():  # <1>
            for imgs, labels in loader:
                outputs = model(imgs)
                
                # Gives us the index of the highest output value 
                _, predicted = torch.max(outputs, dim=1) # <2>
                total += labels.shape[0]  # <3>
                correct += int((predicted == labels).sum())  # <4>

        # Comparing the predicted class
        print("Accuracy {}: {:.2f}".format(name , correct / total))

validate(model, train_loader, val_loader)

Accuracy train: 0.10
Accuracy val: 0.10


In [28]:
# Save our model
torch.save(model.state_dict(), data_path + 'Actual vs predicted.pt')

In [29]:
# Problem 2

# resnet101 function instantiate a 101-layer convolutional 
# neural network, creating an instance of the network
from torchvision import models

resnet = models.resnet101(pretrained=True)


Downloading: "https://download.pytorch.org/models/resnet101-63fe2227.pth" to /root/.cache/torch/hub/checkpoints/resnet101-63fe2227.pth


  0%|          | 0.00/171M [00:00<?, ?B/s]

In [30]:
# to avoid hardcoding numbers in the definition of the model, 
# we can easily pass a parameter to init and parameterize the width

class NetWidth(nn.Module):
    def __init__(self, n_chans1=32):
        super().__init__()
        self.n_chans1 = n_chans1
        self.conv1 = nn.Conv2d(3, n_chans1, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(n_chans1, n_chans1 // 2, kernel_size=3,
                               padding=1)
        self.fc1 = nn.Linear(8 * 8 * n_chans1 // 2, 32)
        self.fc2 = nn.Linear(32, 2)
        
    def forward(self, x):
        out = F.max_pool2d(torch.tanh(self.conv1(x)), 2)
        out = F.max_pool2d(torch.tanh(self.conv2(out)), 2)
        out = out.view(-1, 8 * 8 * self.n_chans1 // 2)
        out = torch.tanh(self.fc1(out))
        out = self.fc2(out)
        return out

In [31]:
model = NetWidth(n_chans1=32).to(device=device)
optimizer = optim.SGD(model.parameters(), lr=1e-2)
loss_fn = nn.CrossEntropyLoss()

training_loop(
    n_epochs = 300,
    optimizer = optimizer,
    model = model,
    loss_fn = loss_fn,
    train_loader = train_loader,
)

all_acc_dict["width"] = validate(model, train_loader, val_loader)

KeyboardInterrupt: ignored

In [31]:
# we can look at how many parameters our model has 
sum(p.numel() for p in model.parameters())

62006

In [32]:
# adding Regularization

# The most popular regularization terms of this kind are:
# a) L2 regularization: which is the sum of squares of all weights in the model
# b) L1 regularization: which is the sum of the absolute values of all weights in the model


def training_loop_l2reg(n_epochs, optimizer, model, loss_fn,
                        train_loader):
    for epoch in range(1, n_epochs + 1):
        loss_train = 0.0
        for imgs, labels in train_loader:
            imgs = imgs.to(device=device)
            labels = labels.to(device=device)
            outputs = model(imgs)
            loss = loss_fn(outputs, labels)

            l2_lambda = 0.001
            l2_norm = sum(p.pow(2.0).sum()
                          for p in model.parameters())  # <1>
            loss = loss + l2_lambda * l2_norm

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            loss_train += loss.item()
        if epoch == 1 or epoch % 10 == 0:
            print('{} Epoch {}, Training loss {}'.format(
                datetime.datetime.now(), epoch,
                loss_train / len(train_loader)))

In [33]:
# Dropout: a Simple Way to Prevent Neural Networks from Overfitting

# zero out a random fraction of outputs from neurons across the 
# network, where the randomization happens at each training iteration (epoch).

class NetDropout(nn.Module):
    def __init__(self, n_chans1=32):
        super().__init__()
        self.n_chans1 = n_chans1
        self.conv1 = nn.Conv2d(3, n_chans1, kernel_size=3, padding=1)
        self.conv1_dropout = nn.Dropout2d(p=0.3)
        self.conv2 = nn.Conv2d(n_chans1, n_chans1 // 2, kernel_size=3,
                               padding=1)
        self.conv2_dropout = nn.Dropout2d(p=0.3)
        self.fc1 = nn.Linear(8 * 8 * n_chans1 // 2, 32)
        self.fc2 = nn.Linear(32, 2)
        
    def forward(self, x):
        out = F.max_pool2d(torch.tanh(self.conv1(x)), 2)
        out = self.conv1_dropout(out)
        out = F.max_pool2d(torch.tanh(self.conv2(out)), 2)
        out = self.conv2_dropout(out)
        out = out.view(-1, 8 * 8 * self.n_chans1 // 2)
        out = torch.tanh(self.fc1(out))
        out = self.fc2(out)
        return out

In [48]:
model = NetDropout(n_chans1=32).to(device=device)
optimizer = optim.SGD(model.parameters(), lr=1e-2)
loss_fn = nn.CrossEntropyLoss()

training_loop(
    n_epochs = 100,
    optimizer = optimizer,
    model = model,
    loss_fn = loss_fn,
    train_loader = train_loader,
)
all_acc_dict["dropout"] = validate(model, train_loader, val_loader)

2022-03-31 01:58:21.036537 Epoch 1, Training loss 2.3056179541151236
2022-03-31 01:59:58.072109 Epoch 10, Training loss 2.305627961902667
2022-03-31 02:01:43.976867 Epoch 20, Training loss 2.305614198870061
2022-03-31 02:03:28.297602 Epoch 30, Training loss 2.3056134744678314
2022-03-31 02:05:14.788056 Epoch 40, Training loss 2.3055988348963314
2022-03-31 02:07:01.183922 Epoch 50, Training loss 2.3056497912272773
2022-03-31 02:08:47.085894 Epoch 60, Training loss 2.305632536063719
2022-03-31 02:10:30.837483 Epoch 70, Training loss 2.305589220407979
2022-03-31 02:12:14.718800 Epoch 80, Training loss 2.305627863730311
2022-03-31 02:14:00.470581 Epoch 90, Training loss 2.3056345479872524
2022-03-31 02:15:46.445126 Epoch 100, Training loss 2.3056202614703754


RuntimeError: ignored

In [34]:
# Batch Normalization: rescale the inputs of the activations, the 
# natural location is after the linear transformation

class NetBatchNorm(nn.Module):
    def __init__(self, n_chans1=32):
        super().__init__()
        self.n_chans1 = n_chans1
        self.conv1 = nn.Conv2d(3, n_chans1, kernel_size=3, padding=1)
        self.conv1_batchnorm = nn.BatchNorm2d(num_features=n_chans1)
        self.conv2 = nn.Conv2d(n_chans1, n_chans1 // 2, kernel_size=3, 
                               padding=1)
        self.conv2_batchnorm = nn.BatchNorm2d(num_features=n_chans1 // 2)
        self.fc1 = nn.Linear(8 * 8 * n_chans1 // 2, 32)
        self.fc2 = nn.Linear(32, 2)
        
    def forward(self, x):
        out = self.conv1_batchnorm(self.conv1(x))
        out = F.max_pool2d(torch.tanh(out), 2)
        out = self.conv2_batchnorm(self.conv2(out))
        out = F.max_pool2d(torch.tanh(out), 2)
        out = out.view(-1, 8 * 8 * self.n_chans1 // 2)
        out = torch.tanh(self.fc1(out))
        out = self.fc2(out)
        return out

In [35]:
from torchvision.models.resnet import resnet101
# Adding a skip connection is adding the output of the first layer in the 
# forward function to the input of the third layer.

# we’re using the output of the first activations as inputs to the last, in 
# addition to the standard feed-forward path. 
# This is also referred to as identity mapping.

class ResNet(nn.Module):
    def __init__(self, n_chans1=32):
        super().__init__()
        self.n_chans1 = n_chans1
        self.conv1 = nn.Conv2d(3, n_chans1, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(n_chans1, n_chans1 // 2, kernel_size=3,
                               padding=1)
        self.conv3 = nn.Conv2d(n_chans1 // 2, n_chans1 // 2,
                               kernel_size=3, padding=1)
        self.fc1 = nn.Linear(4 * 4 * n_chans1 // 2, 32)
        self.fc2 = nn.Linear(32, 2)
        
    def forward(self, x):
        out = F.max_pool2d(torch.relu(self.conv1(x)), 2)
        out = F.max_pool2d(torch.relu(self.conv2(out)), 2)
        out1 = out
        out = F.max_pool2d(torch.relu(self.conv3(out)) + out1, 2)
        out = out.view(-1, 4 * 4 * self.n_chans1 // 2)
        out = torch.relu(self.fc1(out))
        out = self.fc2(out)
        return out

In [ ]:
model = NetBatchNorm(n_chans1=32).to(device=device)
optimizer = optim.SGD(model.parameters(), lr=1e-2)
loss_fn = nn.CrossEntropyLoss()

training_loop(
    n_epochs = 100,
    optimizer = optimizer,
    model = model,
    loss_fn = loss_fn,
    train_loader = train_loader,
)
all_acc_dict["batch_norm"] = validate(model, train_loader, val_loader)

2022-03-31 03:21:08.701156 Epoch 1, Training loss 2.3056050676214115
2022-03-31 03:24:30.284577 Epoch 10, Training loss 2.3056050676214115
2022-03-31 03:28:17.334967 Epoch 20, Training loss 2.3056050676214115
2022-03-31 03:32:02.894605 Epoch 30, Training loss 2.3056050676214115
2022-03-31 03:35:47.850218 Epoch 40, Training loss 2.3056050676214115
2022-03-31 03:39:34.047274 Epoch 50, Training loss 2.3056050676214115


In [35]:
# Depth of a skip connection

class NetDepth(nn.Module):
    def __init__(self, n_chans1=32):
        super().__init__()
        self.n_chans1 = n_chans1
        self.conv1 = nn.Conv2d(3, n_chans1, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(n_chans1, n_chans1 // 2, kernel_size=3,
                               padding=1)
        self.conv3 = nn.Conv2d(n_chans1 // 2, n_chans1 // 2,
                               kernel_size=3, padding=1)
        self.fc1 = nn.Linear(4 * 4 * n_chans1 // 2, 32)
        self.fc2 = nn.Linear(32, 2)
        
    def forward(self, x):
        out = F.max_pool2d(torch.relu(self.conv1(x)), 2)
        out = F.max_pool2d(torch.relu(self.conv2(out)), 2)
        out = F.max_pool2d(torch.relu(self.conv3(out)), 2)
        out = out.view(-1, 4 * 4 * self.n_chans1 // 2)
        out = torch.relu(self.fc1(out))
        out = self.fc2(out)
        return out

In [ ]:
model = NetDepth(n_chans1=32).to(device=device)
optimizer = optim.SGD(model.parameters(), lr=1e-2)
loss_fn = nn.CrossEntropyLoss()

training_loop(
    n_epochs = 300,
    optimizer = optimizer,
    model = model,
    loss_fn = loss_fn,
    train_loader = train_loader,
)
all_acc_dict["depth"] = validate(model, train_loader, val_loader)

In [39]:
# First, in init, we create nn.Sequential containing 
# a list of ResBlock instances.
class ResNet10(nn.Module):
    def __init__(self, n_chans1=32, n_blocks=10):
        super().__init__()
        self.n_chans1 = n_chans1
        self.conv1 = nn.Conv2d(3, n_chans1, kernel_size=3, padding=1)
        self.resblocks = nn.Sequential(
            *(n_blocks * [ResBlock(n_chans=n_chans1)]))
        self.fc1 = nn.Linear(8 * 8 * n_chans1, 32)
        self.fc2 = nn.Linear(32, 2)
    
    # Then, in forward, we just call the sequential to 
    # traverse the 100 blocks and generate the output      
    def forward(self, x):
        out = F.max_pool2d(torch.relu(self.conv1(x)), 2)
        out = self.resblocks(out)
        out = F.max_pool2d(out, 2)
        out = out.view(-1, 8 * 8 * self.n_chans1)
        out = torch.relu(self.fc1(out))
        out = self.fc2(out)
        return out

In [41]:
# Building a very deep network for 10 blocks
class ResBlock(nn.Module):
    def __init__(self, n_chans):
        super(ResBlock, self).__init__()
        self.conv = nn.Conv2d(n_chans, n_chans, kernel_size=3,
                              padding=1, bias=False)  # <1>
        self.batch_norm = nn.BatchNorm2d(num_features=n_chans)
        torch.nn.init.kaiming_normal_(self.conv.weight,
                                      nonlinearity='relu')  # <2>
        torch.nn.init.constant_(self.batch_norm.weight, 0.5)
        torch.nn.init.zeros_(self.batch_norm.bias)

    def forward(self, x):
        out = self.conv(x)
        out = self.batch_norm(out)
        out = torch.relu(out)
        return out + x

In [46]:
model = ResNet10(n_chans1=32).to(device=device)
optimizer = optim.SGD(model.parameters(), lr=1e-2)
loss_fn = nn.CrossEntropyLoss()

training_loop(
    n_epochs = 100,
    optimizer = optimizer,
    model = model,
    loss_fn = loss_fn,
    train_loader = train_loader,
)
all_acc_dict["res"] = validate(model, train_loader, val_loader)

2022-03-31 01:39:31.802599 Epoch 1, Training loss 2.3056176275853306
2022-03-31 01:41:06.839810 Epoch 10, Training loss 2.3056189974250696
2022-03-31 01:42:51.616465 Epoch 20, Training loss 2.3056162376233074
2022-03-31 01:44:36.075252 Epoch 30, Training loss 2.305585544737404
2022-03-31 01:46:21.551305 Epoch 40, Training loss 2.30561593334998
2022-03-31 01:48:06.559410 Epoch 50, Training loss 2.305594876599129
2022-03-31 01:49:52.768990 Epoch 60, Training loss 2.305610506430916
2022-03-31 01:51:38.874851 Epoch 70, Training loss 2.3056237185397723
2022-03-31 01:53:26.075474 Epoch 80, Training loss 2.305588917354184
2022-03-31 01:55:13.041994 Epoch 90, Training loss 2.3056118000498818
2022-03-31 01:57:01.171802 Epoch 100, Training loss 2.3055936524935086


RuntimeError: ignored

In [59]:
model = ResNet10(n_chans1=32, n_blocks=10).to(device=device)
optimizer = optim.SGD(model.parameters(), lr=3e-3)
loss_fn = nn.CrossEntropyLoss()

training_loop(
    n_epochs = 300,
    optimizer = optimizer,
    model = model,
    loss_fn = loss_fn,
    train_loader = train_loader,
)
all_acc_dict["res deep"] = validate(model, train_loader, val_loader)

2022-03-31 00:00:58.557931 Epoch 1, Training loss 2.3056028337429857
2022-03-31 00:03:00.324160 Epoch 10, Training loss 2.3056074420509436
2022-03-31 00:05:14.386450 Epoch 20, Training loss 2.305589846942736
2022-03-31 00:07:30.564429 Epoch 30, Training loss 2.3056129674472468
2022-03-31 00:09:42.771106 Epoch 40, Training loss 2.3056331223539073
2022-03-31 00:11:52.683181 Epoch 50, Training loss 2.305641632555696
2022-03-31 00:14:00.991080 Epoch 60, Training loss 2.3056412532811277
2022-03-31 00:16:08.719574 Epoch 70, Training loss 2.30560118432545
2022-03-31 00:18:15.053387 Epoch 80, Training loss 2.3056298780929096
2022-03-31 00:20:21.397239 Epoch 90, Training loss 2.305649608907187
2022-03-31 00:22:29.761594 Epoch 100, Training loss 2.305594175672897
2022-03-31 00:24:37.608714 Epoch 110, Training loss 2.3056264789513006
2022-03-31 00:26:44.996525 Epoch 120, Training loss 2.30562821251657
2022-03-31 00:28:52.468198 Epoch 130, Training loss 2.305630338466381
2022-03-31 00:31:00.512806

NameError: ignored

In [ ]:
trn_acc = [v['train'] for k, v in all_acc_dict.items()]
val_acc = [v['val'] for k, v in all_acc_dict.items()]

width =0.3
plt.bar(np.arange(len(trn_acc)), trn_acc, width=width, label='train')
plt.bar(np.arange(len(val_acc))+ width, val_acc, width=width, label='val')
plt.xticks(np.arange(len(val_acc))+ width/2, list(all_acc_dict.keys()),
           rotation=60)
plt.ylabel('accuracy')
plt.legend(loc='lower right')
plt.ylim(0.7, 1)
plt.savefig('accuracy_comparison.png', bbox_inches='tight')
plt.show()